In [1]:
import numpy as np
import random

In [2]:
class stateClass:
    def __init__(self,state, K, reward = -1):
        self.K = K
        self.isStart = self.isStart(state)
        self.isEnd = self.isTerminal(state)
        self.state = state
        self.reward = reward
    
        if (self.state == (self.K-1,self.K-1)):
            self.reward = 2*(self.K-1)
            
    def isTerminal(self, state):
        if (state==(self.K-1,self.K-1)):
            return True
        return False
    
    def isStart(self, state):
        if (state==(0,0)):
            return True
        return False
      
    def getStates(self):
        return (self.state[0],self.state[1])


    def getReward(self):
        if self.state == (self.K-1,self.K-1):
            return 2*(self.K-1)
#         elif self.state == LOSE_STATE:
#             return -1
        else:
            return -1

# epsilon greedy policy

In [12]:
class Grid_ep():
    def __init__(self,K):
        self.K = K
#         self.grid = np.zeros((self.K,self.K))
        self.action_value_state = np.zeros((self.K,self.K))
#         self.policy = np.chararray((self.K,self.K)).decode("utf-8")
        self.policy = [["" for i in range(self.K)] for i in range(self.K)]
        
    def q(self, alpha=0.01, num_episodes=300, gamma=0.9,epsilon=0.2):
        for ep in range(num_episodes):
            init = [i for i in range(self.K)]
            m = random.choice(init)
            n = random.choice(init)
            state=stateClass((m,n),self.K)
            while True:
                action = self.take_epsilon_greedy_action(state.getStates(),gamma,epsilon)
                next_state = self.step(state.getStates(),action)
                next_state_ = stateClass(next_state,self.K)
                reward = next_state_.getReward()

                self.action_value_state[state.getStates()[0]][state.getStates()[1]] += alpha*(reward+gamma*self.action_value_state[next_state[0]][next_state[1]]-self.action_value_state[state.getStates()[0]][state.getStates()[1]])
                self.policy[state.getStates()[0]][state.getStates()[1]] = action
                state=next_state_
                if action == 'T':
                    break

        return self.action_value_state,self.policy

    def take_epsilon_greedy_action(self,state, gamma,epsilon):
        if np.random.random() <= epsilon:
            return random.choice(self.getPossibleActions(state))
        else:

            dct={}
            possible_actions = self.getPossibleActions(state)
            for i, val in enumerate(possible_actions):
                if val == ['T']:
                    continue
                next_state = self.step(state,val)
                v_next = self.action_value_state[next_state[0]][next_state[1]]
                next_state = stateClass((next_state[0],next_state[1]),self.K)
                reward = next_state.reward
                v_new = reward + gamma*v_next
                dct[val]=v_new

            valist = [val for val in dct.values()]
            max_value = max(valist)
            self.action_value_state[state[0]][state[1]] = max_value
            best_action = self.get_best_action(dct,max_value)
            return best_action


    def step(self, state,action):
        i = state[0]
        j = state[1]
        if action == 'S': # NORTH
            if i == self.K-1:
                return None
            else:
                i+=1        
          
        elif action == 'E':#EAST
            if j == self.K-1:
                return None
            else:
                j+=1
                        
        elif action == 'N':#SUD
            if i == 0:
                return None
            else:
                i-=1
                                      
        elif action == 'W':#WEST
            if j == 0:
                return None
            else:
                j-=1
            
        return (i,j)
    
    def getPossibleActions(self, state):
        
        if state[0]==0:
            if state[1]==0:
                return ['E','S']
            elif state[1]==self.K-1:
                return ['S','W']
            else:
                return ['W','E','S']

        elif state[0]==self.K - 1:
            if state[1]==0:
                return ['N','E']
            elif state[1]==self.K-1:
                return ['T']
            
            else:
                return ['N','E','W']

        elif state[1]==0:
            return ['S','N','E']
        
        elif state[1]==self.K-1:
            return ['N','S','W']

        else:
            return ['N','E','S','W']
        
    def get_best_action(self,d,val):
        for key, value in d.items():
            if val == value:
                return key
            

In [4]:
k = int(input("Enter k the size of the gridWorld: "))

Enter k the size of the gridWorld: 12


In [13]:
grid = Grid_ep(k)

In [14]:
action_value_state,policy = grid.q()

In [15]:
policy

[['E', 'E', 'S', 'S', 'S', 'W', 'E', 'E', 'S', 'E', 'S', 'W'],
 ['S', 'S', 'W', 'E', 'S', 'W', 'S', 'S', 'S', 'W', 'S', 'W'],
 ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'N', 'W', 'W'],
 ['S', 'S', 'E', 'S', 'W', 'N', 'N', 'E', 'S', 'E', 'S', 'W'],
 ['E', 'S', 'E', 'E', 'S', 'S', 'W', 'S', 'E', 'N', 'S', 'S'],
 ['E', 'S', 'W', 'E', 'E', 'S', 'S', 'S', 'S', 'W', 'S', 'W'],
 ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W'],
 ['N', 'S', 'N', 'E', 'S', 'E', 'E', 'E', 'E', 'S', 'S', 'S'],
 ['N', 'E', 'S', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'W'],
 ['E', 'E', 'E', 'E', 'S', 'E', 'E', 'S', 'S', 'W', 'E', 'S'],
 ['E', 'N', 'E', 'E', 'S', 'S', 'E', 'S', 'E', 'S', 'W', 'S'],
 ['N', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'T']]

In [16]:
def print_policy():
    init = [i for i in range(12)]
    m = random.choice(init)
    n = random.choice(init)
    l = []
    start = (m,n)
    print(f"starting from {start} follow this path")
    l.append(policy[m][n])
    while(policy[m][n]!='T'):
        if policy[m][n]=='E':
            n+=1
            l.append(policy[m][n])
        elif policy[m][n]=='W':
            n-=1
            l.append(policy[m][n])
        elif policy[m][n]=='S':
            m+=1
            l.append(policy[m][n])
        elif policy[m][n]=='N':
            m-=1
            l.append(policy[m][n])
    return l

In [17]:
road = print_policy()
road

starting from (7, 3) follow this path


['E', 'S', 'E', 'E', 'E', 'E', 'S', 'S', 'E', 'S', 'E', 'E', 'T']

# greedy policy

In [11]:
#without epsilon greedy
class Grid():
    def __init__(self,K):
        self.K = K
        self.action_value_state = np.zeros((self.K,self.K))
        self.policy = [["" for i in range(self.K)] for i in range(self.K)]
        
    def q(self, alpha=0.01, num_episodes=300, gamma=0.9):
        for ep in range(num_episodes):
            init = [i for i in range(self.K)]
            m = random.choice(init)
            n = random.choice(init)
            state=stateClass((m,n),self.K)
            while True:
                action = self.take_greedy_action(state.getStates())
                next_state = self.step(state.getStates(),action)
                next_state_ = stateClass(next_state,self.K)
                reward = next_state_.reward
                self.action_value_state[state.getStates()[0]][state.getStates()[1]] += alpha*(reward+gamma*self.action_value_state[next_state[0]][next_state[1]]-self.action_value_state[state.getStates()[0]][state.getStates()[1]])
                self.policy[state.getStates()[0]][state.getStates()[1]] = action
                state=next_state_
                if action == 'T':
                    break

        return self.action_value_state,self.policy

    def take_greedy_action(self,state, gamma=0.9):
        dct={}
        possible_actions = self.getPossibleActions(state)
        for i, val in enumerate(possible_actions):
            if val == ['T']:
                continue
            next_state = self.step(state,val)
            v_next = self.action_value_state[next_state[0]][next_state[1]]
            next_state = stateClass((next_state[0],next_state[1]),self.K)
            reward = next_state.reward
            v_new = reward + gamma*v_next
            dct[val]=v_new

        valist = [val for val in dct.values()]
        max_value = max(valist)
        self.action_value_state[state[0]][state[1]] = max_value
        best_action = self.get_best_action(dct,max_value)
        return best_action


    def step(self, state,action):
        i = state[0]
        j = state[1]
        if action == 'S': # south
            if i == self.K-1:
                return None
            else:
                i+=1        
          
        elif action == 'E':#EAST
            if j == self.K-1:
                return None
            else:
                j+=1
                        
        elif action == 'N':#north
            if i == 0:
                return None
            else:
                i-=1
                                      
        elif action == 'W':#WEST
            if j == 0:
                return None
            else:
                j-=1
            
        return (i,j)
    
    def getPossibleActions(self, state):
        
        if state[0]==0:
            if state[1]==0:
                return ['E','S']
            elif state[1]==self.K-1:
                return ['S','W']
            else:
                return ['W','E','S']

        elif state[0]==self.K - 1:
            if state[1]==0:
                return ['N','E']
            elif state[1]==self.K-1:
                return ['T']
            
            else:
                return ['N','E','W']

        elif state[1]==0:
            return ['S','N','E']
        
        elif state[1]==self.K-1:
            return ['N','S','W']

        else:
            return ['N','E','S','W']
        
    def get_best_action(self,d,val):
        for key, value in d.items():
#             print("the dict",d)
            if val == value:
#                 print("the action chosen",key)
                return key
            

In [18]:
grid = Grid_ep(k)
action_value_state,policy = grid.q()

In [19]:
policy

[['E', 'E', 'S', 'E', 'E', 'S', 'S', 'S', 'E', 'E', 'S', 'W'],
 ['E', 'S', 'E', 'E', 'E', 'E', 'S', 'S', 'E', 'S', 'S', 'W'],
 ['E', 'E', 'S', 'E', 'E', 'E', 'S', 'W', 'S', 'S', 'S', 'S'],
 ['E', 'E', 'S', 'N', 'S', 'S', 'S', 'E', 'S', 'S', 'S', 'S'],
 ['N', 'W', 'E', 'S', 'S', 'S', 'S', 'N', 'E', 'E', 'S', 'S'],
 ['E', 'S', 'N', 'S', 'E', 'S', 'E', 'E', 'E', 'E', 'S', 'S'],
 ['E', 'S', 'E', 'S', 'W', 'S', 'E', 'E', 'S', 'N', 'S', 'S'],
 ['S', 'E', 'E', 'S', 'E', 'E', 'E', 'S', 'E', 'E', 'E', 'S'],
 ['S', 'N', 'E', 'S', 'W', 'E', 'E', 'S', 'N', 'E', 'S', 'S'],
 ['E', 'E', 'S', 'W', 'E', 'S', 'S', 'S', 'W', 'S', 'S', 'S'],
 ['E', 'E', 'E', 'S', 'E', 'E', 'S', 'E', 'E', 'E', 'E', 'S'],
 ['E', 'N', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'T']]

In [20]:
print_policy()

starting from (10, 0) follow this path


['E', 'E', 'E', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'T']